In [73]:
import numpy as np 
import pandas as pd
import requests
from datetime import datetime
import calendar
import matplotlib.pyplot as plt
import math

In [75]:
contracts = pd.read_csv("/home/xikron/Projects/misc/data/signalTest/cA.csv")
contracts_test = contracts[contracts["Start Date"].str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)]
contracts_test["Start Date"] = pd.to_datetime(contracts_test["Start Date"])
contracts_test = contracts_test.sort_values(by="Start Date")

In [76]:
keys= []
date_ranges = [
    ("2010-01-01", "2011-01-01", "c10"),
    ("2011-01-01", "2012-01-01", "c11"),
    ("2012-01-01", "2013-01-01", "c12"),
    ("2013-01-01", "2014-01-01", "c13"),
    ("2014-01-01", "2015-01-01", "c14"),
    ("2015-01-01", "2016-01-01", "c15"),
]

def filter (start_date, end_date, label):
    globals()[label] = contracts_test[
        (contracts_test["Start Date"] >= pd.Timestamp(start_date)) & 
        (contracts_test["Start Date"] < pd.Timestamp(end_date))
    ].reset_index(drop=True, inplace=False).drop_duplicates(subset=['Start Date', 'Recipient Name'])
    keys.append(label)

for start, end, label in date_ranges:
    filter(start, end, label)

In [77]:
globals().get('c10').head()

,internal_id,Award ID,Recipient Name,Award Amount,Total Outlays,Description,Contract Award Type,def_codes,COVID-19 Obligations,COVID-19 Outlays,...,Infrastructure Outlays,Awarding Agency,Awarding Sub Agency,Start Date,End Date,recipient_id,prime_award_recipient_id,awarding_agency_id,agency_slug,generated_internal_id
0,32056385,TIRNO99D000010151,GD,31037339.31,NaN,FY09&10 BSM APPROP AWARD OF O&M '10 TO,DO,NaN,NaN,NaN,...,NaN,Department of the Treasury,Internal Revenue Service,2010-01-01,2011-12-31,dec98d0f-5225-096c-98f0-004844714034-C,NaN,456.0,department-of-the-treasury,CONT_AWD_TIRNO99D000010151_2050_TIRNO99D00001_...
1,4852212,1003,PG,75129752.49,NaN,RESALE - DETERGENT,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Defense Commissary Agency,2010-01-01,2010-03-31,f5ee0c1f-c5bd-0134-19b2-edb6fa728811-C,NaN,1173.0,department-of-defense,CONT_AWD_1003_9700_HDEC0110G3895_9700
2,37308889,W15P7T10CG409,GD,38613194.28,NaN,AN UNDEFINITIZED CONTRACT ACTION TO PROCURE CH...,DCA,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Army,2010-01-04,2011-03-23,fc6c95c4-c4df-e1d8-e534-7c27ff6bbe87-C,NaN,1173.0,department-of-defense,CONT_AWD_W15P7T10CG409_9700_-NONE-_-NONE-
3,37308991,W15P7T10CS406,LMT,37099833.75,NaN,W15P7T-10-C-S406 PROCUREMENT FOR THREE PERSIS...,DCA,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Army,2010-01-05,2011-01-04,1c900490-88f7-8dd4-6ef3-af0672962d06-C,NaN,1173.0,department-of-defense,CONT_AWD_W15P7T10CS406_9700_-NONE-_-NONE-
4,23732977,HR001110C0042,LMT,37685591.84,NaN,THE PURPOSE OF THE NATIONAL CYBER RANGE (NCR) ...,DCA,NaN,NaN,NaN,...,NaN,Department of Defense,Defense Contract Management Agency,2010-01-08,2011-10-31,ea75ddee-2d00-ac7f-5e72-6f42d7c88c8c-C,NaN,1173.0,department-of-defense,CONT_AWD_HR001110C0042_9700_-NONE-_-NONE-


In [78]:
def setLinkEod (ticker, start, end):
    return"https://financialmodelingprep.com/api/v3/historical-price-full/"+ticker+"?from="+start+"&to="+end+"&apikey=26srycwxWrFIhEuaZwic6mBdx7f4VjGT"

In [136]:
def getUnderlyingEOD(ticker, start, end):
    preContract = requests.get(url=setLinkEod(ticker, start, end)).json()
    try:
       
        if 'historical' in preContract:
            return pd.DataFrame([entry['close'] for entry in preContract['historical']], columns=['close'])
        else:
            print("Key 'historical' not found in preContract!")
            return pd.DataFrame()  
    except Exception as e:
        print(f"Error processing data: {e}")
        return pd.DataFrame()  



In [137]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

def getTradingDay(contractDate, prev, isForward):
    uBday = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    contractDate = pd.Timestamp(contractDate)
    tradingDay = contractDate + prev*uBday if isForward else contractDate - prev*uBday
    return tradingDay.strftime('%Y-%m-%d')


In [81]:
def calculate_rsi(close, window=14):
    delta = close.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window).mean()
    avg_loss = pd.Series(loss).rolling(window=window).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

In [94]:
def calculate_signals(data):
    data = data.copy()
    data['log_returns'] = np.log(data['close'] / data['close'].shift(1))
    data['ewma2'] = data['close'].ewm(span=2).mean()
    data['ewma5'] = data['close'].ewm(span=5).mean()
    data['ewma10'] = data['close'].ewm(span=10).mean()
    data['volatility_3'] = data['log_returns'].rolling(3).std()
    data['volatility_5'] = data['log_returns'].rolling(5).std()
    data['moving_avg2'] = data['close'].rolling(2).mean()
    data['moving_avg3'] = data['close'].rolling(3).mean()
    data['moving_avg4'] = data['close'].rolling(4).mean()
    data['rsi'] = calculate_rsi(data['close'])
    return data

In [108]:
len(globals().get('c10')["Recipient Name"].unique())

63

In [ ]:
def contractImpact(year_keys, daysBefore, daysAfter):
    global growths, growth, contracts_analyzed  

    growth = 0  
    contracts_analyzed = 0  
    growths = {}  

    for key in year_keys:
        df = globals().get(key)[:100]

        for i in range(len(df)):
            ticker = df.iloc[i]['Recipient Name']
            contractStart = df.iloc[i]['Start Date']
            preC = getUnderlyingEOD(ticker, getTradingDay(contractStart, daysBefore, False), contractStart.strftime('%Y-%m-%d'))
            if (preC.empty):
                continue
            postC = getUnderlyingEOD(ticker, contractStart.strftime('%Y-%m-%d'), getTradingDay(contractStart, daysAfter, True))

            preSig = calculate_signals(preC) 
            postSig = calculate_signals(postC)

            if preC['close'].mean() < postC['close'].mean():
                growth += 1

            for sig in preSig.columns:
                if preSig[sig].mean() < postSig[sig].mean():
                    if sig not in growths:
                        growths[sig] = 1
                    else:
                        growths[sig] += 1

            contracts_analyzed += 1

        print (f"Signal Testing summary for {year_keys} when looking {daysBefore} days pre contract and {daysAfter} post contract start date")
        print(f"{growth} out of {contracts_analyzed} showed an increase in mean closing price for the year {key}")

    print("\nSignal Growth Summary:")
    for sig, count in growths.items():
        print(f"Signal '{sig}' showed an increase in {count} contracts")

In [ ]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [
        executor.submit(contractImpact, ['c14'], 20, 3), 
        executor.submit(contractImpact, ['c14'], 20, 10),  
        executor.submit(contractImpact, ['c14'], 20, 20),  
        executor.submit(contractImpact, ['c14'], 20, 30),  
    ]

Key 'historical' not found in preContract!
Key 'historical' not found in preContract!
Key 'historical' not found in preContract!
Key 'historical' not found in preContract!
Key 'historical' not found in preContract!
Key 'historical' not found in preContract!
Key 'historical' not found in preContract!
Key 'historical' not found in preContract!
Key 'historical' not found in preContract!


KeyError: 'close'